Importing libraries

In [3]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np

attempt: detecting Black Pieces using RGB thresholding


In [6]:

# load image
img = cv2.imread("data\\all\\IMG_0299_JPG.rf.7d92f1ba0929b7fc5e77848b80148fc6.jpg")

# # Apply Gaussian blur
# blurred_img = cv2.GaussianBlur(img, (5, 5), 0)


# Apply thresholding
threshold = 40
thresholded_img = cv2.inRange(img, (0, 0, 0), (threshold, threshold, threshold))


# Apply opening
kernel = np.ones((5, 5), np.uint8)
opened_img = cv2.morphologyEx(thresholded_img, cv2.MORPH_OPEN, kernel)

# Apply closing
kernel = np.ones((7, 7), np.uint8)
closed_img = cv2.morphologyEx(opened_img, cv2.MORPH_CLOSE, kernel)

# Show final processed image
plt.imshow(cv2.cvtColor(closed_img, cv2.COLOR_BGR2RGB))
plt.title("Processed Image")
plt.axis('off')
plt.show()


In [17]:
# Find contours
contours, _ = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out small noise
min_area = 2000  # Adjust based on your image
bounding_boxes = []
for contour in contours:
    if cv2.contourArea(contour) > min_area:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))

# Draw bounding boxes on original image 
img_with_bounding_boxes = img.copy()
for x, y, w, h in bounding_boxes:
    cv2.rectangle(img_with_bounding_boxes, (x - 15, y - 15), (x + w + 15, y + h + 15), (0, 255, 0), 2)

# Show image with bounding boxes
plt.imshow(cv2.cvtColor(img_with_bounding_boxes, cv2.COLOR_BGR2RGB))
plt.title("Image with Bounding Boxes")
plt.axis('off')
plt.show()

Attempt: trying to understand the rgb values of white pieces


In [4]:
img_path = 'data\\train\\0b4ba28f0c759a11750a6430649b52e3_jpg.rf.c840c64b42f1edd63fd63024fed5ec8b.jpg'
img = cv2.imread(img_path)


# Split the channels
r, g, b = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Set up the histogram properties
colors = ('red', 'green', 'blue')
channels = [r, g, b]

# Plot histograms
plt.figure(figsize=(10, 5))
for i, (channel, color) in enumerate(zip(channels, colors)):
    hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
    plt.plot(hist, color=color, label=f'{color.upper()} channel')

plt.title("RGB Histogram")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.legend()
plt.show()

img_path = 'test.jpg'
img = cv2.imread(img_path)


# Split the channels
r, g, b = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Set up the histogram properties
colors = ('red', 'green', 'blue')
channels = [r, g, b]

# Plot histograms
plt.figure(figsize=(10, 5))
for i, (channel, color) in enumerate(zip(channels, colors)):
    hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
    plt.plot(hist, color=color, label=f'{color.upper()} channel')

plt.title("RGB Histogram")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.legend()
plt.show()




attempt: detect white pieces using RGB thresholding

In [5]:
img_path = 'data\\train\\IMG_0291_JPG.rf.a72d476f25b25f6ef3f768e5876c2a55.jpg' 
img = cv2.imread(img_path)

# Define the thresholds for each channel
lower_bound = np.array([75, 70, 100])
upper_bound = np.array([180, 200, 200])

# Apply the threshold
thresholded_img = cv2.inRange(img, lower_bound, upper_bound)

# Opening
kernel = np.ones((3, 3), np.uint8)
thresholded_img = cv2.morphologyEx(thresholded_img, cv2.MORPH_OPEN, kernel)

# Show the original and thresholded image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(thresholded_img, cmap='gray')
plt.title("Thresholded Image")
plt.axis('off')
plt.show()



attempt: HSV thresholding


In [58]:
# Load image
img = cv2.imread("data\\all\\IMG_0299_JPG.rf.7d92f1ba0929b7fc5e77848b80148fc6.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Convert to HSV color space
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

h, s, v = cv2.split(hsv)
channels = [h, s, v]

h_hist = cv2.calcHist([hsv], [0], None, [180], [0, 180])
s_hist = cv2.calcHist([hsv], [1], None, [256], [0, 256])
v_hist = cv2.calcHist([hsv], [2], None, [256], [0, 256])

# Plot histograms
plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(h_hist, color='red')
plt.title("Hue Histogram")
plt.xlabel("Hue")
plt.ylabel("Frequency")

plt.subplot(1, 3, 2)
plt.plot(s_hist, color='green')
plt.title("Saturation Histogram")
plt.xlabel("Saturation")
plt.ylabel("Frequency")

plt.subplot(1, 3, 3)
plt.plot(v_hist, color='blue')
plt.title("Value Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()



hsv thresholding for white pieces

In [59]:
# Define the lower and upper bounds 
lower = np.array([20, 60, 0])
upper = np.array([50, 255, 255])

# Create a mask for the specified Hue range
mask = cv2.inRange(hsv, lower, upper)

# Display mask
plt.imshow(mask, cmap='gray')
plt.title("Mask")
plt.axis('off')
plt.show()

# Apply the mask to the original image
result_img = cv2.bitwise_and(img, img, mask=mask)

# opening with 5x3 kernel
# kernel = np.ones((5, 5), np.uint8)
# result_img = cv2.morphologyEx(result_img, cv2.MORPH_OPEN, kernel)

# Show the processed image
plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
plt.title("Processed Image")
plt.axis('off')
plt.show()


hsv thresholding for black pieces

In [61]:
# Define the lower and upper bounds 
lower = np.array([35, 0, 0])
upper = np.array([180, 80, 80])

# Create a mask for the specified Hue range
mask = cv2.inRange(hsv, lower, upper)

# Opening
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)


# Display mask
plt.imshow(mask, cmap='gray')
plt.title("Mask")
plt.axis('off')
plt.show()



test: applying bounding box



In [ ]:

# Assuming 'mask' contains your binary image (the one shown)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area to get only the chess pieces
min_area = 1000  # Adjust this value to filter out the small noise spots
chess_pieces = []
bounding_boxes = []

# Visualization image
visualization = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  # Convert to BGR for colored boxes

for contour in contours:
    area = cv2.contourArea(contour)
    if area < min_area:
        continue
        
    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    bounding_boxes.append((x, y, w, h))
    
    # Crop the piece from the original mask (or from your color image if available)
    piece = mask[y:y+h, x:x+w]
    chess_pieces.append(piece)
    
    # Draw rectangle on visualization image
    cv2.rectangle(visualization, (x, y), (x+w, y+h), (0, 255, 0), 2)

print(f"Found {len(chess_pieces)} chess pieces")

# Display results
plt.imshow(cv2.cvtColor(visualization, cv2.COLOR_BGR2RGB))
plt.title("Detected Chess Pieces")
plt.axis('off')
plt.show()